In [ ]:
# Attempt to run the full dataset through Integrated Gradients on GPU

In [ ]:
import torch

In [ ]:
# Python class to define the architecture, taken straight from the classifier/pytorch_fish_classifier file

## this function is here because there used to be (and still are) issues with ReLU layers
def reluToInplaceFalse(model):
  for name, child in model.named_children():
    if isinstance(child, nn.ReLU):
      setattr(child, 'inplace', False)
    else:
      reluToInplaceFalse(child)

from torchvision.transforms.transforms import RandomRotation

class Classifier(torch.nn.Module):

  def __init__(self, backbone='resnet', multi_backbone = True, device ="cuda:0",dropout_rate = 0.2, do_augmentation = False):
    super().__init__()
    self.multi_backbone = multi_backbone

    if backbone == "vgg19":
      backbone = torchvision.models.vgg19(pretrained=True)
      self.out_channels = 25088
      
    elif backbone == "resnet18":
      backbone = torchvision.models.resnet18(pretrained=True)
      self.out_channels = 512

    elif backbone == "resnet50":
      backbone = torchvision.models.resnet50(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b1":
      backbone = torchvision.models.efficientnet_b1(pretrained=True)
      self.out_channels = 1280

    elif backbone == "Efficientnet b3":
      backbone = torchvision.models.efficientnet_b3(pretrained=True)
      self.out_channels = 1536

    elif backbone == "Efficientnet b5":
      backbone = torchvision.models.efficientnet_b5(pretrained=True)
      self.out_channels = 2048

    elif backbone == "Efficientnet b7":
      backbone = torchvision.models.efficientnet_b7(pretrained=True)
      self.out_channels = 2560
      
    # Disabling inplace ReLu becasuse GradCam doesn't work it enabled
    reluToInplaceFalse(backbone)
     
    modules = list(backbone.children())[:-1]
    self.do_augmentation = do_augmentation

    if self.do_augmentation:
      self.augmentation = nn.Sequential(transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomPerspective(0.2),
                                        RandomRotation(20),
                                        transforms.RandomAutocontrast())

    if self.multi_backbone:
      self.backbone1 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone2 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone3 = nn.Sequential(*copy.deepcopy(modules)).to(device)
      self.backbone4 = nn.Sequential(*copy.deepcopy(modules)).to(device)
    else:
      self.backbone =  nn.Sequential(*modules).to(device)


     

    self.fc1 = nn.Sequential(nn.Dropout(dropout_rate),
                              nn.Linear(self.out_channels, 128),
                              nn.ReLU(),
                              nn.Dropout(dropout_rate)) #TODO: Experiment with BN and Dropout

    # 512 features in, 3 features out
    self.fc = nn.Sequential(nn.Linear(512, 3))                  #TODO: L2 Regularization
     
  def forward(self, x, is_training = True):
    if self.do_augmentation and is_training:
      imgs = [self.augmentation(x[:,i]) for i in range(4)] #list of 4 images
    else:
      imgs = [x[:,i] for i in range(4)] #list of 4 images
      #imgs = [x[i] for i in range(4)]

    if self.multi_backbone:
        # feed each image into a backbone
      encodings = [self.fc1(torch.flatten(self.backbone1(imgs[0]),1)),
                   self.fc1(torch.flatten(self.backbone2(imgs[1]),1)),
                   self.fc1(torch.flatten(self.backbone3(imgs[2]),1)),
                   self.fc1(torch.flatten(self.backbone4(imgs[3]),1))]
    else:
      encodings = [self.fc1(self.backbone(img).squeeze()) for img in imgs]

    # modify to return an array with the largest encoding
    
    #raw_result = self.fc(torch.cat(encodings,1))
    #fixed_result = torch.clone(raw_result)
    #for idx, row in enumerate(raw_result):
    #    ones = torch.ones(3, dtype=torch.long)
    #    max_val = row.max()
    #    fixed_result[idx] = torch.where(row < max_val, 0, ones)
                  
    #return fixed_result.type(torch.int64)
    return self.fc(torch.cat(encodings,1))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instantiate the classifier using the following weights
# If your directory structure does not resemble the following, you may need to 
model = torch.load("drive/MyDrive/Fish Attribution/" + 'model1e-05500.5_april_4.pt', map_location="cpu")
model.eval()
model.zero_grad()

In [ ]:
print(torch.cuda.is_available())
# check that cuda is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

True
cuda:0


In [ ]:
# Put the model onto the GPU for faster evaluation
model.to(device)

Classifier(
  (backbone1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): Re

In [ ]:
# Load the raw fish images
# WARNING: The following WILL use a large amount of RAM
import numpy as np

X_PATH = "/content/drive/Shareddrives/Exploding Gradients/x_train_b_cropped.npy"
Y_PATH= "/content/drive/Shareddrives/Exploding Gradients/y_train_b.npy"

x = np.load(X_PATH)
y = np.load(Y_PATH)

# 285 instances, currently the shapes are off, see next cell for fix
print(x.shape)
print(y.shape)

(285, 4, 130, 750, 3)
(285, 1)


In [ ]:
# Conver to PyTorch tensors from raw numpy arrays and
# fix the dimensions
tensor_x = torch.Tensor(x) 
tensor_y = torch.Tensor(y).long()

tensor_x = torch.swapaxes(tensor_x,2,4)
tensor_x = torch.swapaxes(tensor_x,3,4)

In [ ]:
# There are 258 pieces of data, with 4 images per instance
# and each image being full color (RGB) with 130x750 pixels
# (The original fish images are of larger dimension but
# an autocropper was created to narrow down the focus)
tensor_x.shape

torch.Size([285, 4, 3, 130, 750])

In [ ]:
# Create a PyTorch dataloader to feed data to the model
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

attribution_ds = TensorDataset(tensor_x ,nn.functional.one_hot(tensor_y,3)) 

# Each time this is called, 40 pieces of data (40x4 = 160) are provided
# This was figured out in a bit of trial-and-error fashion because
# you have a limited about of onboard GPU RAM. 40 seems to be the maximum
# it's happy with and any Colab Pro instance goes into the red at this point
attribution_dl = DataLoader(attribution_ds,40,shuffle = True)

# Delete the raw numpy arrays from RAM, there's no need for them
# now that they exist as PyTorch tensors
del x,y

In [ ]:
# Install Captum for Feature Attribution
!pip install -q captum

from captum.attr import *

In [ ]:
ig = IntegratedGradients(model)

In [ ]:
# Save each batch of attributions
all_attributions = []
# you do 40 at a time here
batch_num = 1
iter_attribution_dl = iter(attribution_dl)
for images, labels in iter_attribution_dl:
  print(f"Processing batch #{batch_num}") # for 285 instances 40 at a time, should process 8 batches
  batch_num += 1
  # set internal batch size (must be at least = to number of images per iteration)
  num_images_per_batch = len(images)
  # send images to the GPU
  images = images.to(device)
  # create new class-representative labels
  new_labels = []
  for t in labels.squeeze():
    if torch.equal(t, torch.tensor([1,0,0])):
      new_labels.append(0)
    elif torch.equal(t, torch.tensor([0,1,0])):
      new_labels.append(1)
    elif torch.equal(t, torch.tensor([0,0,1])):
      new_labels.append(2)
  new_labels = torch.tensor(new_labels,device=device)
  # get the attributions
  ## important arguments:
  ## n_steps: number of steps that should be executed for Integration
  ## internal_batch_size: 
  attributions = ig.attribute(images, target=new_labels, baselines = (images * 0), n_steps = 50, internal_batch_size = num_images_per_batch)
  # Convert the attribution data to CPU and save it in the list
  all_attributions.append(attributions.cpu())
  del images, labels, attributions
  torch.cuda.empty_cache()

Processing batch #1
Processing batch #2
Processing batch #3
Processing batch #4
Processing batch #5
Processing batch #6
Processing batch #7
Processing batch #8


In [ ]:
# all batches have 40 elements...
all_attributions[0].shape

torch.Size([40, 4, 3, 130, 750])

In [ ]:
# Except for the last batch which falls short...
all_attributions[-1].shape

torch.Size([5, 4, 3, 130, 750])

In [ ]:
all_attributions_tensor = torch.cat(all_attributions, 0)

In [ ]:
# the attribution data should have the exact same dimensions as
# the original data, this will be critical for generating the overlays!
all_attributions_tensor.shape

torch.Size([285, 4, 3, 130, 750])

In [ ]:
# The UMAP generation is performed in a separate notebook
torch.save(all_attributions_tensor, "drive/MyDrive/Fish Attribution/" +  "x_train_b_cropped_integrated_gradients.tensor")